# From Lammpstrj to Vertex Dataframe

In this script, I will convert the lammpstrj data to a vertex dataframe. This script will be usefull for all the simulations that I will run in a future. Since is a way to preprocess the data. 

### 1. Reading data and convert to dataframe

In [1]:
import magcolloids as mgc
import os
import sys
import numpy as np
import pandas as pd
from multiprocessing import Pool
import tqdm.auto as tqdm

sys.path.insert(0, '../icenumerics')

import icenumerics as ice
import matplotlib.pyplot as plt

In [2]:
data_directory = "/home/carolina/CairoLattice_ACI_PROJECT/Scripts/Output_Script-1.4"

In [3]:
def do_everything(exp_entry):
    
    e = exp_entry[1].e 
    l = exp_entry[1].l
    
    load_obj = mgc.trj_lazyread(os.path.join(data_directory,
                                             "ACI_CairoLattice_50mT-3000s+cte_l%u_exp%u.lammpstrj"%(l,e)),
                                                output = ["x","y","z","mux","muy","muz"])    
    bnd = load_obj.get_bounds()
    trj = load_obj.read_trj()
    col_trj = ice.get_ice_trj(trj,bnd, atom_types = 1, trap_types = [2,3])
    
    # Infer Vertices
    frames = col_trj.index.get_level_values("frame").unique()

    v = ice.vertices()
    v.trj_to_vertices(col_trj.loc[frames[::10]])
    
    # Remove the vertices near the boundaries
    
    x_min = 95
    y_min = -75

    width = 700
    hight = -700

    x_max = x_min+width
    y_max = y_min+hight
    
    Ca = v.vertices.loc[(v.vertices['x']<x_max) & (v.vertices['x']>x_min)]
    Cairo = Ca.loc[(Ca['y']>y_max) & (Ca['y']<y_min)]
    
    #plt.plot(Cairo.loc[0].x,Cairo.loc[0].y, 'o')
    
    Cairo.index.names = ['frame', 'id']
    
    Cairo.to_csv(os.path.join(data_directory, "ACI_CairoLattice_50mT-3000s+cte_l%u_exp%u.dat"%(l,e)), sep="\t")


In [4]:
e =  np.arange(0,10)
l =  [10]
L, E = np.meshgrid(l,e)
experiments = pd.DataFrame({"e":E.flatten(),"l":L.flatten()})
experiments

,e,l
0,0,10
1,1,10
2,2,10
3,3,10
4,4,10
5,5,10
6,6,10
7,7,10
8,8,10
9,9,10


In [ ]:
if __name__ ==  '__main__': 
    num_processors = 2
    p=Pool(processes = num_processors)
        
    list(tqdm.tqdm(p.imap(do_everything, experiments.iterrows()), total=len(experiments)));